# yt-napari 

```
pip install napari[pyQT5] yt-napari dask[distributed]
```

or if you care about QT versions

```
pip install napari[pyQT6] yt-napari 
```

one other plugin:

```
pip install napari-simpleitk-image-processing
```


In [ ]:
import napari 
from yt_napari.viewer import Scene
import yt 

viewer = napari.Viewer()

In [ ]:

ds = yt.load("IsolatedGalaxy/galaxy0030/galaxy0030")

yt_scene = Scene()

left_edge = ds.domain_center - ds.arr([10, 10, 10], 'kpc')
right_edge = ds.domain_center + ds.arr([10, 10, 10], 'kpc')
yt_scene.add_region(viewer,
                    ds,
                    ("enzo", "Density"),
                    left_edge = left_edge,
                    right_edge = right_edge,
                    resolution=(600, 600, 600),
                    colormap="magma",
                    contrast_limits=(3.4, 6.4))

yt_scene.add_region(viewer,
                    ds,
                    ("enzo", "Temperature"),
                    left_edge = left_edge,
                    right_edge = right_edge,
                    resolution=(300, 300, 300),
                    colormap="yellow",
                    contrast_limits=(4., 6.),
                    opacity=0.6)

Can also: 

* load from JSON file
* load from GUI
  
https://yt-napari.readthedocs.io/en/latest/quickstart.html#loading-a-json-file-from-the-napari-gui

## Loading timeseries slices with dask

In [ ]:
from dask.distributed import Client

In [ ]:
c = Client(n_workers=2, threads_per_worker=1)

In [ ]:
from yt_napari import timeseries

viewer.layers.clear() 

slc = timeseries.Slice(("enzo", "Density"), "x", resolution=(400, 400))
file_pattern = "enzo_tiny_cosmology/DD????/DD????"
timeseries.add_to_viewer(viewer,
                         slc, 
                         file_pattern=file_pattern, 
                         load_as_stack=True,
                         use_dask=True,
                         contrast_limits=(-1, 2),
                         colormap = 'magma',
                         name="Lazy density")

## The napai plugin ecosystem

https://yt-napari.readthedocs.io/en/latest/tutorials.html
* Interactive visualization with the wider plugin ecosystem
* Image segmentation and classification with additional plugins

https://www.napari-hub.org/

In [ ]:
viewer.layers.clear()

In [ ]:
ds = yt.load_sample("Enzo_64")

yt_scene = Scene()

left_edge = ds.domain_center - ds.arr([10, 10, 10], 'kpc')
right_edge = ds.domain_center + ds.arr([10, 10, 10], 'kpc')
yt_scene.add_region(viewer,
                    ds,
                    ("gas", "density"),
                    left_edge = ds.domain_left_edge,
                    right_edge = ds.domain_right_edge,
                    rescale=True,
                    resolution=(256, 256, 256),                    
                    colormap="magma",)

yt_scene.add_region(viewer,
                    ds,
                    ("gas", "density"),
                    left_edge = ds.domain_left_edge,
                    right_edge = ds.domain_right_edge,
                    resolution=(512, 512, 512),                    
                    colormap="viridis",)render plane manipulator to move the rendering plane and explore the image. Open the render 

simple-itk:

* Otsu multiple thresholds (drop to 128 bins, bump levels to 5)
* Label statistics